In [31]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [34]:
f = open("/content/drive/My Drive/Malayalam_NLP/Data/pos_tagged_data/Malayalam_Tagged_Data.txt","r")
data = f.read()
print(len(data))
data = data.split(".        \RD_PUNC")
all_sent = []
for x in data:
    x = x.split("\n")
    v = []
    for y in x:
        if y != "":
            y = y.replace("\\","")
            y = y.split("        ")
            v.append( (y[0],y[1]) )
    v.append(('.','RD_PUNC'))
    all_sent.append(v)
del data
sentences = all_sent
del all_sent
print(len(sentences))

sentences = sentences[:10000]
print(len(sentences))


TypeError: ignored

In [4]:
print(sentences[0])
other_words = []
for x in sentences:
  for y in x:
    if y[0] not in other_words:
      other_words.append(y[0])


[('മഹാത്മാഗാന്ധി', 'N_NNP'), ('രാഷ്ട്രപിതാവ്', 'N_NN'), ('ലക്ഷ്യങ്ങളെല്ലാം', 'N_NN'), ('കര്\u200d്മ്മഫലത്തിലെത്തിച്ച', 'V_VM_VNF'), ('മഹാൻ', 'N_NN'), ('.', 'RD_PUNC')]


In [6]:
print(len(other_words))

76711


In [0]:
sentence =[]
tags = []
count = 0
c = 0

for x in sentences:
  s = []
  t = []
  for y in x:
    if len(y)>=2:
      s.append(y[0])
      t.append(y[1])
      
  sentence.append((s,t))

  

In [22]:
print(sentence[0])

(['മഹാത്മാഗാന്ധി', 'രാഷ്ട്രപിതാവ്', 'ലക്ഷ്യങ്ങളെല്ലാം', 'കര്\u200d്മ്മഫലത്തിലെത്തിച്ച', 'മഹാൻ', '.'], ['N_NNP', 'N_NN', 'N_NN', 'V_VM_VNF', 'N_NN', 'RD_PUNC'])


In [23]:
data = []
tagged_data = []
for x in sentence:
  data.append(x[0])
  tagged_data.append(x[1])

print(data[10])
print(tagged_data[10])

['അതിനു', 'പുറമെ', 'പ്രത്യക്ഷമായിത്തന്നെ', 'വിദ്യാഭ്യാസം', 'സ്വയംപര്യാപ്തമാക്കാന്\u200dവേണ്ടി', 'വ്യവസായാഭ്യസനവുമായി', 'ഒന്നിച്ചുതുടര്\u200dന്നുകൊണ്ടു', 'പോകേണ്ടുന്നതിന്\u200dറെ', 'ആവശ്യകത', 'കൂടിഈ', 'രാജ്യത്തുണ്ട്', '.']
['DM_DMD', 'PSP', 'RB', 'N_NN', 'V_VM_VNF', 'RB', 'V_VM_VNF', 'N_NN', 'N_NN', 'DM_DMD', 'V_VAUX', 'RD_PUNC']


In [0]:

from sklearn.model_selection import train_test_split
 
(train_sentences, 
 test_sentences, 
 train_tags, 
 test_tags) = train_test_split(data, tagged_data, test_size=0.2)

words, tags = set([]), set([])
 
for s in train_sentences:
    for w in s:
        words.add(w.lower())
 
for ts in train_tags:
    for t in ts:
        tags.add(t)
 
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding

In [25]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []
 
for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)
 
for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)
 
for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])
 
for s in test_tags:
    test_tags_y.append([tag2index[t] for t in s])
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])
 

[51983, 24896, 50822, 33916, 56286, 13676, 43706, 9641, 62991, 48556, 10743, 8254, 42844]
[46830, 44209, 26797, 40981, 1, 45328, 1, 17755, 23259, 35649, 1, 1, 60949, 42844]
[34, 9, 34, 35, 34, 34, 21, 34, 34, 34, 21, 2, 33]
[23, 34, 34, 34, 32, 32, 32, 34, 34, 9, 34, 28, 2, 33]


In [26]:
MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH)  # 271

from keras.preprocessing.sequence import pad_sequences
 
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

TypeError: ignored

In [0]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
 
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
 
model.summary()
 






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 62, 128)           4197120   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 62, 512)           788480    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 62, 36)            18468     
_________________________________________________________________
activation_1 (Activation)    (None, 62, 36)            0         
Total params: 5,004,068
Trainable params: 5,004,068
Non-trainable params: 0
_________________________________________________________________


In [0]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)
 
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print(cat_train_tags_y[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [0]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=10, validation_split=0.2, verbose=1)

Train on 6400 samples, validate on 1600 samples
Epoch 1/10
6400/6400 [==============================] - 82s 13ms/step - loss: 0.0032 - acc: 0.9996 - val_loss: 0.2073 - val_acc: 0.9587
Epoch 2/10
6400/6400 [==============================] - 82s 13ms/step - loss: 0.0027 - acc: 0.9997 - val_loss: 0.2139 - val_acc: 0.9599
Epoch 3/10
6400/6400 [==============================] - 81s 13ms/step - loss: 0.0023 - acc: 0.9997 - val_loss: 0.2090 - val_acc: 0.9587
Epoch 4/10
6400/6400 [==============================] - 82s 13ms/step - loss: 0.0020 - acc: 0.9998 - val_loss: 0.2181 - val_acc: 0.9589
Epoch 5/10
6400/6400 [==============================] - 82s 13ms/step - loss: 0.0017 - acc: 0.9998 - val_loss: 0.2231 - val_acc: 0.9595
Epoch 6/10
6400/6400 [==============================] - 82s 13ms/step - loss: 0.0015 - acc: 0.9998 - val_loss: 0.2225 - val_acc: 0.9590
Epoch 7/10
6400/6400 [==============================] - 82s 13ms/step - loss: 0.0014 - acc: 0.9999 - val_loss: 0.2254 - val_acc: 0.9590


In [0]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print(f"{model.metrics_names[1]}: {scores[1] * 100}")   # acc: 99.09751977804825
#https://nlpforhackers.io/lstm-pos-tagger-keras/

2000/2000 [==============================] - 6s 3ms/step
acc: 96.04677419662475


In [0]:
	
test_samples = [
    "എന്നാൽ ഏതാനം സമയം ചിത്രങ്ങളിൽ ശ്രദ്ധിച്ചിരുന്ന രവിവർമ്മക്ക്‌ അത്‌ വളരെ എളുപ്പം മനസ്സിലാക്കാൻ കഴിഞ്ഞു .".split()
#    "I was running every day for a month .".split()
]
print(test_samples)


[['എന്നാൽ', 'ഏതാനം', 'സമയം', 'ചിത്രങ്ങളിൽ', 'ശ്രദ്ധിച്ചിരുന്ന', 'രവിവർമ്മക്ക്\u200c', 'അത്\u200c', 'വളരെ', 'എളുപ്പം', 'മനസ്സിലാക്കാൻ', 'കഴിഞ്ഞു', '.']]


In [0]:
test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)
 
test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)
predictions = model.predict(test_samples_X)

def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences


print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

[[    1     1 15897     1     1     1     1 18872 17202     1   308  1621
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0]]
[['N_NN', 'N_NN', 'N_NN', 'N_NN', 'N_NN', 'N_NN', 'N_NN', 'RP_INTF', 'RB', 'N_NN', 'V_VM_VF', 'RD_PUNC', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-']]


In [0]:
print(len(sentence))
textData = sentence[2500:]
training_data = sentence

5000


In [0]:


def prepare_sequence(seq, to_ix):
    """Input: takes in a list of words, and a dictionary containing the index of the words
    Output: a tensor containing the indexes of the word"""
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [0]:
word_to_ix = {} # This is the word dictionary which will contain the index to each word

for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix.keys():
            word_to_ix[word] = len(word_to_ix)
for word in other_words:
    if word not in word_to_ix.keys():
            word_to_ix[word] = len(word_to_ix)

# print(word_to_ix) # Just have a look at what it contains

tag_to_ix = {'N_NNP': 0, 'N_NN': 1, 'V_VM_VNF': 2, 'RD_PUNC': 3, 'QT_QTF': 4, 'RB': 5, 'PR_PRF': 6, 'JJ': 7, 'PR_PRP': 8, 'V_VAUX': 9, 'QT_QTO': 10, 'RP_INTF': 11, 'RD_RDF': 12, 'DM_DMD': 13, 'DM_DMR': 14, 'CC_CCD': 15, 'PSP': 16, 'V_VM_VF': 17, 'N_NST': 18, 'CC_CCS': 19, 'RP_NEG': 20, 'V_VM_VINF': 21, 'DM_DMQ': 22, 'PR_PRL': 23, 'V_VM': 24, 'RP_RPD': 25, 'PR_PRC': 26, 'V_VN': 27, 'RP_INJ': 28, 'QT_QTC': 29, 'PR_PRQ': 30,'CC_CCS_UT': 31,'RD_UNK':32}
   #{"DET": 0, "NN": 1, "V": 2, "ADJ": 3, "ADV": 4, "PRP": 5, "PRN": 6} # This dictionary contains the indices of the tags

EMBEDDING_DIM = 64
HIDDEN_DIM = 64

In [0]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, target_size):
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [0]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix.keys()), len(tag_to_ix.keys()))

# Define the loss function as the Negative Log Likelihood loss (NLLLoss)
loss_function = nn.NLLLoss()

# We will be using a simple SGD optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)

# The test sentence

    
print("Training Started")
for epoch in range(300):
    print("epoch :", epoch)
    for sentence, tags in training_data[:2500]:
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        tag_scores = model(sentence_in)
        
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
print("Training Finished!!!\nAgain testing on unknown data")
with torch.no_grad():
    for seq in [seq1, seq2]:
        inputs = prepare_sequence(seq, word_to_ix)
        tag_scores = model(inputs)
        _, indices = torch.max(tag_scores, 1)
        ret = []
        for i in range(len(indices)):
            for key, value in tag_to_ix.items():
                if indices[i] == value:
                    ret.append((seq[i], key))
        print(ret)

Training Started
epoch : 0
epoch : 1
epoch : 2
epoch : 3
epoch : 4
epoch : 5
epoch : 6
epoch : 7
epoch : 8
epoch : 9
epoch : 10
epoch : 11
epoch : 12
epoch : 13
epoch : 14
epoch : 15
epoch : 16
epoch : 17
epoch : 18
epoch : 19
epoch : 20
epoch : 21
epoch : 22
epoch : 23
epoch : 24
epoch : 25
epoch : 26
epoch : 27
epoch : 28
epoch : 29
epoch : 30
epoch : 31
epoch : 32
epoch : 33
epoch : 34
epoch : 35
epoch : 36
epoch : 37
epoch : 38
epoch : 39
epoch : 40
epoch : 41
epoch : 42
epoch : 43
epoch : 44
epoch : 45
epoch : 46
epoch : 47
epoch : 48
epoch : 49
epoch : 50
epoch : 51
epoch : 52
epoch : 53
epoch : 54
epoch : 55
epoch : 56
epoch : 57
epoch : 58
epoch : 59
epoch : 60
epoch : 61
epoch : 62
epoch : 63
epoch : 64
epoch : 65
epoch : 66
epoch : 67
epoch : 68
epoch : 69
epoch : 70
epoch : 71
epoch : 72
epoch : 73
epoch : 74
epoch : 75
epoch : 76
epoch : 77
epoch : 78
epoch : 79
epoch : 80
epoch : 81
epoch : 82
epoch : 83
epoch : 84
epoch : 85
epoch : 86
epoch : 87
epoch : 88
epoch : 89
epo

NameError: ignored

In [0]:
print(len(training_data))

2500


In [0]:
#torch.save(model.state_dict(),'/content/drive/My Drive/NLP Research/s2.model')
torch.save(model.state_dict(),'/content/drive/My Drive/NLP Research/s2.model')

In [0]:
#seq1 = 'മഹാത്മാഗാന്ധി  രാഷ്ട്രപിതാവ് ലക്ഷ്യങ്ങളെല്ലാം കര്\u200d്മ്മഫലത്തിലെത്തിച്ച മഹാൻ .'.split()
with torch.no_grad():
    for seq in textData:
        inputs = prepare_sequence(seq, word_to_ix)
        tag_scores = model(inputs)
        _, indices = torch.max(tag_scores, 1)
        ret = []
        for i in range(len(indices)):
            for key, value in tag_to_ix.items():
                if indices[i] == value:
                    ret.append((seq[i], key))
        print(ret)

TypeError: ignored

In [0]:
sentence2 =[]
tags = []
count = 0
c = 0

for x in sentences[2500:]:
  s = []
  t = []
  for y in x:
    if len(y)>=2:
      s.append(y[0])
      t.append(y[1])
      
  sentence2.append((s,t))

In [0]:
#print(sentence2[0])
testData2 = []
for x in testData[:500]:
  sub = ""
  for y in x[0]:
    sub+=y
    sub+=" "
  testData2.append(sub)

In [0]:
print(testData[1])

IndexError: ignored

In [0]:
seq1 =['അവര്‍ ബക്കററ് പലതവണ താഴേക്കിറക്കിയതാണു് .'.split()] 
#seq1 = []

with torch.no_grad():
      for seq in seq1:

          inputs = prepare_sequence(seq, word_to_ix)
          tag_scores = model(inputs)
          _, indices = torch.max(tag_scores, 1)
          ret = []
          for i in range(len(indices)):
              for key, value in tag_to_ix.items():
                  if indices[i] == value:
                      ret.append((seq[i], key))
          print(ret)
  

NameError: ignored